In [5]:
import requests
from bs4 import BeautifulSoup
import re

In [8]:
url = 'http://www.boxofficemojo.com/movies/?page=intl&id=easya.htm&sort=percoftotal&order=DESC&p=.htm'
def scrape_page(url):
    page = url
    response = requests.get(url)
    try:
        soup = BeautifulSoup(response.text)
        return soup
    except:
        return None

In [9]:
easya = scrape_page(url)

In [83]:
movie_title = ""
movie_data = []

In [95]:
movie_title = easya.find('title').text.split(" (")[0]

In [38]:
header = easya.find_all('td')[4]
international = easya.find_all('table')[7]

In [39]:
header

<td align="center" valign="top"><br/><font face="Verdana" size="6"><b>Easy A</b></font><br/><br/>
<table border="0" cellpadding="0" cellspacing="0"><tr><td>
<center><table bgcolor="#dcdcdc" border="0" cellpadding="4" cellspacing="1" width="95%"><tr bgcolor="#ffffff"><td align="center" colspan="2"><font size="4">Domestic Total Gross: <b>$58,401,464</b></font></td></tr><tr bgcolor="#ffffff"><td valign="top">Distributor: <b><a href="/studio/chart/?studio=screengems.htm">Sony / Screen Gems</a></b></td><td valign="top">Release Date: <b><nobr><a href="/schedule/?view=bydate&amp;release=theatrical&amp;date=2010-09-17&amp;p=.htm">September 17, 2010</a></nobr></b></td></tr><tr bgcolor="#ffffff"><td valign="top">Genre: <b>Romantic Comedy</b></td><td valign="top">Runtime: <b>1 hrs. 32 min.</b></td></tr><tr bgcolor="#ffffff"><td valign="top">MPAA Rating: <b>PG-13</b></td><td valign="top">Production Budget: <b>$8 million</b></td></tr></table> </center></td>
</tr></table></td>

In [101]:
def get_header_value(table, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = table.find(text=re.compile(field_name))
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 
    else:
        return ''

In [106]:
dtg = get_header_value(header,'Domestic Total')
dist = get_header_value(header,'Distributor')
release = get_header_value(header, 'Release Date')
genre = get_header_value(header, 'Genre')
runtime = get_header_value(header,'Runtime')
budget = get_header_value(header, 'Budget')
rating = get_header_value(header,'MPAA Rating')

movie_data.append(dtg)
movie_data.append(dist)
movie_data.append(release)
movie_data.append(genre)
movie_data.append(runtime)
movie_data.append(budget)
movie_data.append(rating)

In [107]:
movie_data

[u'$58,401,464',
 u'Sony / Screen Gems',
 u'September 17, 2010',
 u'Romantic Comedy',
 u'1 hrs. 32 min.',
 u'$8 million',
 u'PG-13']

In [77]:
countries = international.find_all('tr')
foreign_total = []
hungary = []

In [81]:
# do this for foreign_total and for Hungary
# e.g. get_international('Hungary', hungary)
def get_international(country, country_list):
    country_data = ''
    for row in countries:
        data = row.find_all('td')
        if data[0].find(text=country):
            country_data = row
        else:
            break
    #get country name, insert as first element to list
    country_list.append(country_data.find('a').contents[0])
    columns = country_data.find_all('td')[1:]
    for column in columns:
        country_list.append(column.get_text())
    return country_list